In [1]:
import torch
import torch.nn as nn
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)

In [2]:
from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np

In [3]:


# Sample data for sentiment analysis (you can replace with your own task)
sample_data = {
    'texts': [
        "I love this product, it's amazing!",
        "This is terrible, worst purchase ever.",
        "The weather is nice today.",
        "I hate waiting in long queues.",
        "This movie was fantastic and entertaining.",
        "The service was disappointing and slow.",
        "What a beautiful sunset tonight.",
        "I'm frustrated with this situation.",
        "This book is incredibly well written.",
        "The food was bland and overpriced.",
        "I enjoy spending time with friends.",
        "This software is buggy and unreliable.",
        "The concert was absolutely wonderful.",
        "I'm tired of all these problems.",
        "This vacation was the best ever!",
        "The meeting was boring and pointless."
    ],
    'labels': [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]  # 1: positive, 0: negative
}

In [13]:
import torch
import torch.nn as nn
from transformers import (
    AutoTokenizer,
    AutoModel,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding
)

from datasets import Dataset
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np


# Sample data for sentiment analysis (you can replace with your own task)
sample_data = {
    'texts': [
        "I love this product, it's amazing!",
        "This is terrible, worst purchase ever.",
        "The weather is nice today.",
        "I hate waiting in long queues.",
        "This movie was fantastic and entertaining.",
        "The service was disappointing and slow.",
        "What a beautiful sunset tonight.",
        "I'm frustrated with this situation.",
        "This book is incredibly well written.",
        "The food was bland and overpriced.",
        "I enjoy spending time with friends.",
        "This software is buggy and unreliable.",
        "The concert was absolutely wonderful.",
        "I'm tired of all these problems.",
        "This vacation was the best ever!",
        "The meeting was boring and pointless."
    ],
    'labels': [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]  # 1: positive, 0: negative
}


class TransferLearningModel:
    """
    A class to handle transfer learning for text classification
    """

    def __init__(self, model_name="distilbert-base-uncased", num_labels=2):
        """
        Initialize the transfer learning model

        Args:
            model_name: Pre-trained model to use as base
            num_labels: Number of classes for the new task
        """
        self.model_name = model_name
        self.num_labels = num_labels
        self.tokenizer = None
        self.model = None
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


    def load_pretrained_model(self):
        """
        Load the pre-trained model and tokenizer
        """
        print(f"Loading pre-trained model: {self.model_name}")

        # Load tokenizer
        self.tokenizer = AutoTokenizer.from_pretrained(self.model_name)

        # Load model for sequence classification
        # This automatically adds a classification head on top of the base model
        self.model = AutoModelForSequenceClassification.from_pretrained(
            self.model_name,
            num_labels=self.num_labels
        ).to(self.device)

        print(f"Model loaded successfully!")
        print(f"Model has {self.model.num_parameters()} parameters")

    def prepare_dataset(self, texts, labels, max_length=128):
        """
        Prepare dataset for training/evaluation
        """
        def tokenize_function(examples):
            return self.tokenizer(
                examples['text'],
                truncation=True,
                padding=True,
                max_length=max_length,
                return_tensors="pt"
            )

        # Create dataset
        dataset = Dataset.from_dict({
            'text': texts,
            'labels': labels
        })

        # Tokenize
        tokenized_dataset = dataset.map(tokenize_function, batched=True)

        return tokenized_dataset

    def compute_metrics(self, eval_pred):
        """
        Compute metrics for evaluation
        """
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)

        precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average='weighted')
        accuracy = accuracy_score(labels, predictions)

        return {
            'accuracy': accuracy,
            'f1': f1,
            'precision': precision,
            'recall': recall
        }

    def train(self, train_texts, train_labels, eval_texts=None, eval_labels=None):
        """
        Train the model using transfer learning
        """
        print("Preparing training dataset...")
        train_dataset = self.prepare_dataset(train_texts, train_labels)

        eval_dataset = None
        if eval_texts and eval_labels:
            print("Preparing evaluation dataset...")
            eval_dataset = self.prepare_dataset(eval_texts, eval_labels)

        # Data collator
        data_collator = DataCollatorWithPadding(tokenizer=self.tokenizer)

        # Training arguments
        training_args = TrainingArguments(
            output_dir="./transfer_learning_model",
            num_train_epochs=3,
            per_device_train_batch_size=8,
            per_device_eval_batch_size=8,
            warmup_steps=100,
            weight_decay=0.01,
            logging_dir="./logs",
            logging_steps=10,
            eval_strategy="epoch" if eval_dataset else "no",
            save_strategy="epoch",
            load_best_model_at_end=True if eval_dataset else False,
            metric_for_best_model="accuracy" if eval_dataset else None,
            report_to=None,  # Disable wandb
        )

        # Initialize trainer
        trainer = Trainer(
            model=self.model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            tokenizer=self.tokenizer,
            data_collator=data_collator,
            compute_metrics=self.compute_metrics if eval_dataset else None,
        )

        # Train the model
        print("Starting transfer learning training...")
        trainer.train()

        # Save the model
        trainer.save_model("./transfer_learning_model")
        self.tokenizer.save_pretrained("./transfer_learning_model")

        print("Transfer learning completed!")

        return trainer

    def predict(self, texts):
        """
        Make predictions on new texts
        """
        if self.model is None:
            raise ValueError("Model not loaded. Call load_pretrained_model() first.")

        self.model.eval()
        predictions = []

        for text in texts:
            # Tokenize
            inputs = self.tokenizer(
                text,
                return_tensors="pt",
                truncation=True,
                padding=True,
                max_length=128
            )

            # Move inputs to the same device as the model
            inputs = {k: v.to(self.device) for k, v in inputs.items()}

            # Predict
            with torch.no_grad():
                outputs = self.model(**inputs)
                logits = outputs.logits
                predicted_class = torch.argmax(logits, dim=1).item()
                confidence = torch.softmax(logits, dim=1).max().item()

                predictions.append({
                    'text': text,
                    'predicted_class': predicted_class,
                    'confidence': confidence
                })

        return predictions

In [5]:
def demonstrate_different_approaches():
    """
    Demonstrate different transfer learning approaches
    """
    print("=== APPROACH 1: Full Fine-tuning ===")
    # Fine-tune all layers
    model1 = TransferLearningModel("distilbert-base-uncased")
    model1.load_pretrained_model()

    # Split data for training and testing
    train_size = int(0.8 * len(sample_data['texts']))
    train_texts = sample_data['texts'][:train_size]
    train_labels = sample_data['labels'][:train_size]
    test_texts = sample_data['texts'][train_size:]
    test_labels = sample_data['labels'][train_size:]

    trainer1 = model1.train(train_texts, train_labels, test_texts, test_labels)

    print("\n=== APPROACH 2: Feature Extraction (Frozen Base) ===")
    # Freeze base model, only train classification head
    model2 = TransferLearningModel("distilbert-base-uncased")
    model2.load_pretrained_model()

    # Freeze base model parameters
    for param in model2.model.distilbert.parameters():
        param.requires_grad = False

    print("Frozen base model parameters. Only training classification head.")
    trainer2 = model2.train(train_texts, train_labels, test_texts, test_labels)

    return model1, model2

In [15]:
def load_and_test_model(model_path="./transfer_learning_model"):
    """
    Load a saved model and test it
    """
    print(f"\n=== TESTING SAVED MODEL ===")

    # Load the saved model
    model = AutoModelForSequenceClassification.from_pretrained(model_path)
    tokenizer = AutoTokenizer.from_pretrained(model_path)

    # Create model wrapper
    transfer_model = TransferLearningModel()
    transfer_model.model = model.to(transfer_model.device) # Move the loaded model to the correct device
    transfer_model.tokenizer = tokenizer

    # Test with new examples
    test_examples = [
        "This is absolutely wonderful!",
        "I'm really disappointed with this.",
        "The weather looks great today.",
        "This is the worst experience ever.",
        "I love how this works perfectly."
    ]

    predictions = transfer_model.predict(test_examples)

    print("Predictions on new examples:")
    for pred in predictions:
        sentiment = "Positive" if pred['predicted_class'] == 1 else "Negative"
        print(f"Text: '{pred['text']}'")
        print(f"Predicted: {sentiment} (Confidence: {pred['confidence']:.3f})")
        print("-" * 50)

In [7]:


def main():
    """
    Main function to demonstrate transfer learning
    """
    print("Transfer Learning with Language Models")
    print("=" * 50)

    # Check device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print(f"Using device: {device}")

    try:
        # Demonstrate different approaches
        model1, model2 = demonstrate_different_approaches()

        # Test predictions with the first model
        test_texts = [
            "This product is amazing!",
            "I hate this so much.",
            "Pretty decent overall.",
        ]

        print("\n=== MAKING PREDICTIONS ===")
        predictions = model1.predict(test_texts)

        for pred in predictions:
            sentiment = "Positive" if pred['predicted_class'] == 1 else "Negative"
            print(f"Text: '{pred['text']}'")
            print(f"Prediction: {sentiment} (Confidence: {pred['confidence']:.3f})")
            print("-" * 30)

        # Test loading saved model
        load_and_test_model()

    except Exception as e:
        print(f"An error occurred: {e}")
        print("\nMake sure you have the required packages:")
        print("pip install torch transformers datasets scikit-learn")


In [9]:
!pip install torch transformers datasets scikit-learn

In [16]:
if __name__ == "__main__":
    main()

Transfer Learning with Language Models
Using device: cuda
=== APPROACH 1: Full Fine-tuning ===
Loading pre-trained model: distilbert-base-uncased


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model loaded successfully!
Model has 66955010 parameters
Preparing training dataset...


Map:   0%|          | 0/12 [00:00<?, ? examples/s]

Preparing evaluation dataset...


...

Map:   0%|          | 0/4 [00:00<?, ? examples/s]

/tmp/ipython-input-1967886126.py:154: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Starting transfer learning training...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.701456,0.000000,0.000000,0.000000,0.000000
2,No log,0.701281,0.000000,0.000000,0.000000,0.000000
3,No log,0.700966,0.000000,0.000000,0.000000,0.000000


Transfer learning completed!

=== MAKING PREDICTIONS ===
Text: 'This product is amazing!'
Prediction: Negative (Confidence: 0.508)
------------------------------
Text: 'I hate this so much.'
Prediction: Negative (Confidence: 0.505)
------------------------------
Text: 'Pretty decent overall.'
Prediction: Negative (Confidence: 0.500)
------------------------------

=== TESTING SAVED MODEL ===
Predictions on new examples:
Text: 'This is absolutely wonderful!'
Predicted: Negative (Confidence: 0.513)
--------------------------------------------------
Text: 'I'm really disappointed with this.'
Predicted: Negative (Confidence: 0.507)
--------------------------------------------------
Text: 'The weather looks great today.'
Predicted: Negative (Confidence: 0.505)
--------------------------------------------------
Text: 'This is the worst experience ever.'
Predicted: Negative (Confidence: 0.501)
--------------------------------------------------
Text: 'I love how this works perfectly.'
Predicte